In [ ]:
!pip install pandas
!pip install requests

In [ ]:
import pandas as pd
import os
import requests
import time
import re

In [ ]:
import re

def extract_year_period_week(filename):
    """Extracts year, period, and week from a filename like '2024_p1_w1.csv'.

    Args:
        filename: The name of the file.

    Returns:
        A tuple containing the year, period, and week as integers.
    """
    match = re.search(r"(\d+)_p(\d+)_w(\d+)", filename)
    if match:
        year = int(match.group(1))
        period = int(match.group(2))
        week = int(match.group(3))
        return year, period, week
    else:
        return None, None, None  # Handle cases where the pattern is not found

In [ ]:
import pandas as pd

for filename in ['2024_p7_w3.csv']:
    df = pd.read_csv(filename, dtype={'GROWER': str, 'TERRAIN': str})
    year, period, week = extract_year_period_week(filename)

    if year is not None and period is not None and week is not None:
        df['year'] = year  # Add 'Year' column
        df['period'] = period  # Add 'Period' column
        df['week'] = week  # Add 'Week' column
    else:
        print(f"Warning: Could not extract year, period, and week from {filename}")

    # Process the DataFrame further...

In [ ]:
df.head()

In [ ]:
melted_df = pd.melt(df,
                   id_vars=['GROWER',
                             'TERRAIN',
                             'BAGGING',
                             'STEMS',
                             'KLS.A',
                             'BXS.A',
                             'KLS.B',
                             'BXS.B',
                            'startdate','enddate','year','period','week'],
                   value_vars=['WK_10', 'WK_11', 'WK_12', 'WK_13',
                               'WK_14', 'WK_15', 'WK_16', 'WK_17'],
                   var_name='Week_Name',
                   value_name='stem_harvest_week_count')

display(melted_df.head())

In [ ]:
def rename_csv_files(directory):
  """Renames CSV files in a directory from 'p{period}_w{week}.csv' to 'processed_p{period}_w{week}.csv'.

  Args:
    directory: The directory containing the CSV files.
  """
  for filename in os.listdir(directory):
    if filename.endswith(".csv") and filename.startswith("p"):
      base_name = os.path.splitext(filename)[0]  # Get filename without extension
      period, week = base_name[1:].split("_")  # Extract period and week
      new_filename = f"processed_{period}_{week}.csv"
      os.rename(os.path.join(directory, filename), os.path.join(directory, new_filename))
      print(f"Renamed '{filename}' to '{new_filename}'")

In [ ]:
# prompt: from the melted_df, i would need to insert the data from each row to my strapi cms using a http post request

# Replace with your Strapi API endpoint for creating entries
strapi_api_endpoint = "http://127.0.0.1:1337/api/harvests"

# Iterate through the rows of the melted_df
for index, row in melted_df.iterrows():
  # Create a dictionary with the data for your Strapi content type
  data = { "data":{
      "grower": row["GROWER"],
      "terrainid": row["TERRAIN"],
      "bagging": row["BAGGING"],
      "stems": row["STEMS"],
      "class_type_a": row["KLS.A"],
      "box_type_a": row["BXS.A"],
      "class_type_b": row["KLS.B"],
      "box_type_b": row["BXS.B"],
      "stem_harvest_week": row["Week_Name"],
      "stem_harvest_count": row["stem_harvest_week_count"],
      "startdate": row["startdate"],
      "enddate": row["enddate"],
      "period": row["period"],
      "week": row["week"],
      "year": row["year"],
      # Add other fields as needed
  }}
  print(data)
  try:
    # Make a POST request to your Strapi API
    response = requests.post(strapi_api_endpoint, json=data)
    response.raise_for_status()  # Raise an exception for bad status codes

    print(f"Successfully created entry for row {index}: {response.json()}")

  except requests.exceptions.RequestException as e:
    print(f"Error creating entry for row {index}: {e}")
  time.sleep(.3)  # Wait for 1 second before processing the next row


In [ ]:
# prompt: from the melted_df save it to a csv
new_filename = f'processed_{file_name.split('_')[0][1]}_{file_name.split('_')[1].split('.')[0][1]}.csv'
melted_df.to_csv(new_filename, index=False)

This part is a merged code to put all data to a pandas dataframe and load it to RAM

In [ ]:
import os
import pandas as pd
import re

def extract_year_period_week(filename):
    """Extracts year, period, and week from a filename like '2024_p1_w1.csv'.

    Args:
        filename: The name of the file.

    Returns:
        A tuple containing the year, period, and week as integers.
    """
    match = re.search(r"(\d+)_p(\d+)_w(\d+)", filename)
    if match:
        year = int(match.group(1))
        period = int(match.group(2))
        week = int(match.group(3))
        return year, period, week
    else:
        return None, None, None  # Handle cases where the pattern is not found

def process_csv_files(directory):
    """Processes all CSV files in a directory, ingests data into a pandas DataFrame, and melts the DataFrame.

    Args:
        directory: The directory containing the CSV files.

    Returns:
        A pandas DataFrame containing the melted data from all CSV files.
    """
    all_data = []  # List to store data from all files

    for filename in os.listdir(directory):
        if filename.endswith(".csv") and filename.startswith("20"):  # Process only CSV files starting with "20"
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath, dtype={'GROWER': str, 'TERRAIN': str})
            
            year, period, week = extract_year_period_week(filename)
            if year is not None and period is not None and week is not None:
                df['year'] = year  # Add 'Year' column
                df['period'] = period  # Add 'Period' column
                df['week'] = week  # Add 'Week' column
            else:
                print(f"Warning: Could not extract year, period, and week from {filename}")

            # Melt the DataFrame
            melted_df = pd.melt(df,
                               id_vars=['GROWER', 'TERRAIN', 'BAGGING', 'STEMS', 'KLS.A', 'BXS.A', 'KLS.B', 'BXS.B', 'startdate', 'enddate', 'year', 'period', 'week'],
                               value_vars=['WK_10', 'WK_11', 'WK_12', 'WK_13', 'WK_14', 'WK_15', 'WK_16', 'WK_17'],
                               var_name='Week_Name',
                               value_name='stem_harvest_week_count')

            all_data.append(melted_df)  # Append melted data to the list

    # Concatenate all data into a single DataFrame
    final_df = pd.concat(all_data, ignore_index=True)
    return final_df

# Example usage:
directory_path = './data'  # Replace with the actual directory path
final_df = process_csv_files(directory_path)
final_df.head()
# Now you have the final_df containing data from all CSV files in the directory

In [4]:
final_df.head()

,GROWER,TERRAIN,BAGGING,STEMS,KLS.A,BXS.A,KLS.B,BXS.B,startdate,enddate,year,period,week,Week_Name,stem_harvest_week_count
0,A. NOEL,0304,209,149.0,2871.04,214.26,202.37,15.10,02-12-2024,07-12-2024,2024,10,1,WK_10,0.0
1,RABIA,0304,49,0.0,0.00,0.00,0.00,0.00,02-12-2024,07-12-2024,2024,10,1,WK_10,0.0
2,"ALVAREZ, E.",0006,172,51.0,1056.50,78.84,11.02,0.82,02-12-2024,07-12-2024,2024,10,1,WK_10,0.0
3,"ONGKINGCO, J.",0013,110,107.0,2108.69,157.36,38.50,2.87,02-12-2024,07-12-2024,2024,10,1,WK_10,0.0
4,"REYES, P.",0027,67,56.0,1159.98,86.57,39.62,2.96,02-12-2024,07-12-2024,2024,10,1,WK_10,0.0
